In [14]:
from bs4 import BeautifulSoup
import requests
import time
import csv
import pandas as pd

In [2]:
def scrape_main(link):
    '''
    a function that request the webpage and store it in response object. then passing lxml parser to parse over the webpage.
    
    here lxml parser defines the speed to parse the webpage.
    
    if it throws any errors in using lxml parser. just install it by: " pip intall lxml "
    '''
    
    response = requests.get(link)
    return(BeautifulSoup(response.text, 'lxml' ))

In [3]:
#reading first csv file which contains the links

df = pd.read_csv("first_links.csv")

In [4]:
#checking 5 job records
df.head(5)

,Unnamed: 0,source,location,job_link
0,0,internshala,work_from_home,https://internshala.com/internship/detail/mobi...
1,1,internshala,work_from_home,https://internshala.com/internship/detail/cont...
2,2,internshala,work_from_home,https://internshala.com/internship/detail/ente...
3,3,internshala,work_from_home,https://internshala.com/internship/detail/busi...
4,4,internshala,work_from_home,https://internshala.com/internship/detail/grap...


In [5]:
row_heading = ['source', 'location', 'job_link', 'job_title','company_name', 'imp_fields','description_headings','description']

In [6]:
# timestamp as file name by using time library and with prefix as internshala
file_name = 'internshala_second_raw_data_' + time.strftime("%d_%m_%Y_%H_%M_%S") + ".csv" 


# opening file in write mode and connecting csv writer to file
file = open(file_name,'w')
writer = csv.writer(file)


# initially writing header of csv file
writer.writerow(row_heading)


93

In [7]:
# looping over dataframe link column with index value to give job location and source to new data frame
for index,link in enumerate(df.job_link):
    # passing  url to scrape on each job link
    soup = scrape_main(link)
    
    # getting job title by finding unique class name in webpage
    if soup.find('div',{'class':'heading_4_5 profile'}) == None:
        continue
    
    job_title = soup.find('div',{'class':'heading_4_5 profile'}).text.strip()
    
    # getting company name by unique class name of div tag
    company_name = soup.find('div',{'class':'heading_6 company_name'}).text.strip()
    
    
    '''
    # getting important fields in the job posting as list
    fields are:
    1. start date of joining/mode of vacancy
    2. duration of job
    3. incentives/stipend
    4. last date to apply
    5. types of doing job 
    
    
    '''
    imp_fields = []
    for i in soup.find_all('div',{'class':'item_body'}):
        imp_fields.append(i.get_text().strip())
        
    # this list is extracting for further process to do in description to get valuable information.
    description_headings = []
    for i in soup.find_all('div',{'class':'section_heading heading_5_5'}):
        description_headings.append(i.get_text().strip())
        
    # complete description of job
    description = soup.find('div',{'class':'internship_details'}).get_text().strip()
    
    # writing to the server
    writer.writerow([df.source[index], df.location[index], df.job_link[index],job_title,company_name, imp_fields,description_headings, description])

In [8]:
file.close()

In [9]:
# loading extracted csv file to the dataframe
new_df = pd.read_csv(file_name)

In [10]:

new_df.sample(5)

,source,location,job_link,job_title,company_name,imp_fields,description_headings,description
6040,internshala,work_from_home,https://internshala.com/internship/detail/ente...,Enterprise Accounts,Dunzo,"['Starts\xa0immediatelyImmediately', '6 Months...","['About Dunzo', 'About the work from home job/...",About Dunzo \r\n\r\nhttp://dunzo.in/ \r...
7531,internshala,bangalore,https://internshala.com/internship/detail/web-...,Web Development,SKMTECHNOSOLZ Private Limited,"['Starts\xa0immediatelyImmediately', '2 Months...","['About SKMTECHNOSOLZ Private Limited', 'About...",About SKMTECHNOSOLZ Private Limited \r\...
1919,internshala,work_from_home,https://internshala.com/internship/details/pro...,Product Management,Forbes Center For Excellence,"['Starts\xa0immediatelyImmediately', '3 Months...","['About Forbes Center For Excellence', 'About ...",About Forbes Center For Excellence \r\n...
7351,internshala,bangalore,https://internshala.com/internship/detail/rese...,Research,Unacademy,"['Starts\xa0immediatelyImmediately', '3 Months...","['About Unacademy', 'About the internship', 'W...",About Unacademy \r\n\r\nhttps://unacade...
83,internshala,work_from_home,https://internshala.com/internship/detail/busi...,Business Development (Sales),Urban Company,"['Starts\xa0immediatelyImmediately', '1 Month'...","['About Urban Company', 'About the work from h...",About Urban Company \r\n\r\nhttp://urba...


In [11]:
# getting size of csv file
new_df.shape

(8115, 8)

#### To save it into normal csv file just run below cell

In [13]:
new_df.to_csv('second_data.csv')

#### To save file - pipe '|' as delimiter : just run the below cell

In [ ]:
new_df.to_csv('first_pipe_format.csv',sep ='|')